In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [2]:
MODEL_GPT = 'gpt-4o-mini'

## Basic App: Question & Answering from Document

In [3]:
# from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI

In [4]:
# llm = OpenAI()
llm = ChatOpenAI(model=MODEL_GPT)

### Load text file

In [5]:
from langchain.document_loaders import TextLoader

In [6]:
# loader = TextLoader("data/be-good.txt")
loader = TextLoader("../../data/be-good.txt")

In [7]:
document = loader.load()

### Document is loaded as Python list with metadata

In [8]:
print(type(document))

<class 'list'>


In [9]:
print(len(document))

1


In [10]:
print(document[0].metadata)

{'source': '../../data/be-good.txt'}


In [11]:
print(f"You have {len(document)} document.")

You have 1 document.


In [12]:
print(f"Your document has {len(document[0].page_content)} characters")

Your document has 16710 characters


### Split document in small chunks

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [14]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000,
    chunk_overlap=400
)

In [15]:
document_chunks = text_splitter.split_documents(document)

In [16]:
print(f"Now you have {len(document_chunks)} chunks.")

Now you have 8 chunks.


### Convert text chunks in numeric vectors (called "embeddings")

In [17]:
# from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings

In [18]:
embeddings = OpenAIEmbeddings()

### Load embeddings to vector database

In [19]:
from langchain.vectorstores import FAISS

In [20]:
stored_embeddings = FAISS.from_documents(document_chunks, embeddings)

### Create Retrieval Question & Answering Chain

In [21]:
from langchain.chains import RetrievalQA

In [22]:
QA_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=stored_embeddings.as_retriever()
)

### Now we have Question & Answering App

In [23]:
question = """
What is this article about? 
Describe it in less than 100 words.
"""

In [24]:
# QA_chain.run(question)
QA_chain.invoke(question)

{'query': '\nWhat is this article about? \nDescribe it in less than 100 words.\n',
 'result': "The article discusses the importance of creating products that genuinely meet users' needs, emphasizing that startups should focus on making something people want rather than obsessing over their business model initially. It suggests that benevolent intentions in business can lead to success, using examples like Craigslist and Google. The author argues that being good and prioritizing users can serve as a compass for decision-making in startups, ultimately fostering a positive environment that attracts talent and investment. The piece reflects on the intersection of altruism and entrepreneurship."}

In [25]:
question2 = """
And how does it explain how to create somethin people want?
"""

In [26]:
# QA_chain.run(question2)
QA_chain.invoke(question)

{'query': '\nWhat is this article about? \nDescribe it in less than 100 words.\n',
 'result': 'The article discusses the importance of creating products that people want while not overly focusing on immediate profitability. It suggests that startups can operate with benevolent aims, similar to charities, and still achieve success. Examples like Craigslist and Google illustrate how companies can prioritize user needs and genuine growth over profits. The author emphasizes that being good and benevolent can attract talent and support, acting as a guiding principle for decision-making in startups. Overall, it advocates for a user-centric approach in entrepreneurship.'}

## Simple Agent

In [27]:
#!pip install numexpr

In [28]:
from langchain.agents import load_tools
# from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
from langchain.agents import AgentType

# llm=OpenAI()
llm=ChatOpenAI(model=MODEL_GPT)

tool_names = ["llm-math"]
tools = load_tools(tool_names, llm=llm)
tools

[Tool(name='Calculator', description='Useful for when you need to answer questions about math.', func=<bound method Chain.run of LLMMathChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question

In [29]:
from langchain.agents import initialize_agent

agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True,
                         max_iterations=3)

C:\Users\Pavel\AppData\Local\Temp\ipykernel_20560\3660245525.py:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. See LangGraph documentation for more details: https://langchain-ai.github.io/langgraph/. Refer here for its pre-built ReAct agent: https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/
  agent = initialize_agent(tools,


In [30]:
# agent.run("What is 133 by 142?")
agent.invoke("What is 133 by 142?")



> Entering new AgentExecutor chain...
To find the product of 133 and 142, I need to multiply these two numbers together.  
Action: Calculator  
Action Input: 133 * 142  
Observation: Answer: 18886
I now know the final answer.  
Final Answer: 18886

> Finished chain.


{'input': 'What is 133 by 142?', 'output': '18886'}

### Let's make agent fail

In [33]:
# agent.run("Who was the wife of Napoleon Bonaparte?")
agent.invoke("Who was the wife of Napoleon Bonaparte?")



> Entering new AgentExecutor chain...
To answer the question, I need to recall historical information about Napoleon Bonaparte's marriages. I know he was married twice. His first wife was Joséphine de Beauharnais, and his second wife was Marie Louise of Austria. Since the question asks for "the wife" without specifying, it is likely referring to his first and more well-known spouse. 

Action: None needed for calculations; I will provide the answer based on historical knowledge.

Final Answer: Joséphine de Beauharnais was the wife of Napoleon Bonaparte.

> Finished chain.


{'input': 'Who was the wife of Napoleon Bonaparte?',
 'output': 'Joséphine de Beauharnais was the wife of Napoleon Bonaparte.'}

## Custom Agent

In [34]:
# from typing import Optional
# from langchain.tools import BaseTool
# from langchain.callbacks.manager import AsyncCallbackManagerForToolRun
# from langchain.callbacks.manager import CallbackManagerForToolRun

In [35]:
# class CustomSearchTool(BaseTool):
#     name = "article search"
#     description = "useful for when you need to answer questions about our article"

#     def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
#         store = stored_embeddings.as_retriever()
#         docs = store.get_relevant_documents(query)
#         text_list = [doc.page_content for doc in docs]
#         return "\n".join(text_list)

#     async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
#         """Use the tool asynchronously."""
#         raise NotImplementedError("custom_search does not support async")

In [36]:
from langchain.tools import BaseTool
from typing import Optional, Any
from langchain.callbacks.manager import CallbackManagerForToolRun, AsyncCallbackManagerForToolRun

In [37]:
class CustomSearchTool(BaseTool):
    name: str = "article search"  # Add type annotation here
    description: str = "useful for when you need to answer questions about our article"  # Add type annotation here
    
    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        store = stored_embeddings.as_retriever()
        docs = store.get_relevant_documents(query)
        text_list = [doc.page_content for doc in docs]
        return "\n".join(text_list)

    async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")

In [38]:
from langchain.agents import AgentType

tools = [CustomSearchTool()]

agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True, 
    max_iterations=3
)

In [39]:
# agent.run("What is this article about? Describe it in less than 100 words.")
agent.invoke("What is this article about? Describe it in less than 100 words.")



> Entering new AgentExecutor chain...
I need to find out the content of the article to summarize it.  
Action: article search  
Action Input: "What is this article about?"  

C:\Users\Pavel\AppData\Local\Temp\ipykernel_20560\1292071831.py:7: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = store.get_relevant_documents(query)



Observation: Be good
April 2008(This essay is derived from a talk at the 2008 Startup School.)About a month after we started Y Combinator we came up with the
phrase that became our motto: Make something people want.  We've
learned a lot since then, but if I were choosing now that's still
the one I'd pick.Another thing we tell founders is not to worry too much about the
business model, at least at first.  Not because making money is
unimportant, but because it's so much easier than building something
great.A couple weeks ago I realized that if you put those two ideas
together, you get something surprising.  Make something people want.
Don't worry too much about making money.  What you've got is a
description of a charity.When you get an unexpected result like this, it could either be a
bug or a new discovery.  Either businesses aren't supposed to be
like charities, and we've proven by reductio ad absurdum that one
or both of the principles we began with is false.  Or we have a new
idea

{'input': 'What is this article about? Describe it in less than 100 words.',
 'output': 'The article discusses the philosophy behind startup success, emphasizing the importance of creating something people want without initially worrying about profitability. It suggests that benevolent intentions can lead to success and attract talent, using examples like Craigslist and Google to illustrate how businesses can operate with a charitable mindset while still thriving. The author argues that focusing on user needs can guide decision-making and help navigate the complexities of running a startup.'}